## Generating the LIST Dataframes FOR THE KG


In this notebook we are going to generate and save (into disk) a dataframe per CE ontology NODE. 
This is the list of the dataframes saved to file. These dataframes have **ONLY the necessary attributes for later generating the graph, and linking the information between nodes**:

- df_events	
- df_event_category 
- df_event_tags
- df_event_description
- df_event_properties
- df_event_phonenumber	

- df_schedules
- df_schedules_phone_numbers

- df_performances			
- df_performances_properties
- df_performances_descriptions	
- df_performances_links		
			
- df_tickets

- df_places
- df_places_tags
- df_places_properties
- df_places_description
- df_places_loc	
- df_places_pn



We are also generating another intermediate dataframes (those dataframes have 'total' in their names). But those intermediate dataframes will not be saved to disk. 

The idea will be later to read from those dataframes saved to disk to create the knowledge graph. 

In order to create these, we are going to use two dataframes previously calculated:
 - df_news_events
 - df_places
 
 

In [ ]:
import yaml
import string
import copy
from datetime import datetime
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize
from difflib import SequenceMatcher
import pickle
import numpy as np
import collections
from yaml import safe_load

We are going to save the dataframes in a directory called dataframes_final.
#Replace this path with the location you want to save the folder 'dataframes_final'

In [ ]:
#replace this with the location you want to save the folder 'dataframes_final'
!mkdir /content/drive/MyDrive/dataframes_final


mkdir: cannot create directory ‘/content/drive/MyDrive/dataframes_final’: No such file or directory


In [ ]:
dataframes_final="/content/drive/MyDrive/dataframes_final"


## 1. EVENTS DATAFRAME

Going to read the events dataframe and work with it. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.listdir("/content/drive/MyDrive")

### Replace the location with the location of downloaded copy of df_new_events from google drive

In [ ]:
with open("/content/drive/MyDrive/df_new_events","rb") as df_new_events:
    df_new_events=pickle.load(df_new_events)

In [ ]:

df_new_events


,event_id,modified_ts,created_ts,name,sort_name,status,id,schedules,descriptions,website,...,category,properties,ranking_level,ranking_in_level,source_file,period_start,period_end,frequency_event,phone_numbers,alternative_names
0,157884,2021-01-13T18:46:26Z,2007-12-06T17:18:12Z,Väsen,Väsen,live,157884,"[{'start_ts': '2018-04-26T20:00:00+01:00', 'en...","[{'type': 'description.list.default', 'descrip...",http://www.twoforjoy.co.uk,...,Music,{'list:website:comments-end-date': '2013-01-31...,3,2,sample_20171101.json,2017-11-01,2018-05-01,1,NaN,NaN
1,194419,2022-02-02T01:03:54Z,2010-01-25T14:51:46Z,Martin Simpson,Martin Simpson,live,194419,"[{'start_ts': '2018-03-10T19:30:00+00:00', 'en...","[{'type': 'description.list.default', 'descrip...",http://www.martinsimpson.com/,...,Music,{'list:website:comments-end-date': '2020-01-28...,2,1,"sample_20171101.json, sample_20180501.json",2017-11-01,2018-11-01,2,NaN,NaN
2,240818,2018-01-28T12:20:36Z,2011-12-07T11:23:54Z,Burns Night Special,Burns Night Special,live,240818,"[{'start_ts': '2018-01-24T20:30:00+00:00', 'en...","[{'type': 'description.list.default', 'descrip...",NaN,...,Comedy,"{'dropin_event': False, 'booking_essential': F...",3,2,sample_20171101.json,2017-11-01,2018-05-01,1,NaN,NaN
3,345866,2019-06-09T12:51:49Z,2013-03-01T10:33:08Z,Red Raw,Red Raw,live,345866,"[{'start_ts': '2017-11-06T20:30:00+00:00', 'en...","[{'type': 'description.list.default', 'descrip...",NaN,...,Comedy,"{'dropin_event': False, 'booking_essential': F...",3,2,"sample_20171101.json, sample_20180501.json, sa...",2017-11-01,2020-05-01,5,NaN,NaN
4,347164,2020-03-23T07:05:08Z,2013-03-18T13:05:44Z,The Saturday Show,Saturday Show,live,347164,"[{'start_ts': '2017-11-04T21:00:00+00:00', 'en...","[{'type': 'description.list.default', 'descrip...",NaN,...,Comedy,"{'dropin_event': False, 'booking_essential': F...",3,2,"sample_20171101.json, sample_20180501.json, sa...",2017-11-01,2020-11-01,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,1584208,2020-07-28T12:14:54Z,2020-07-22T16:39:58Z,Micro Banger World Cup,Micro Banger World Cup,live,1584208,"[{'start_ts': '2020-10-10T16:00:00+01:00', 'en...","[{'type': 'description.list.default', 'descrip...",NaN,...,Sport,"{'dropin_event': False, 'booking_essential': F...",3,2,sample_20200501.json,2020-05-01,2020-11-01,1,NaN,NaN
2276,1586592,2020-08-04T17:45:27Z,2020-08-03T14:03:15Z,The Magical Highlands Tour,Magical Highlands Tour,live,1586592,"[{'start_ts': '2020-08-12T08:00:00+01:00', 'en...","[{'type': 'description.list.default', 'descrip...",NaN,...,Days out,"{'dropin_event': False, 'booking_essential': F...",3,2,sample_20200501.json,2020-05-01,2020-11-01,1,{'info': '0131 555 5558'},NaN
2278,1595055,2020-09-08T10:53:48Z,2020-09-06T21:24:58Z,Black History Walking Tour of Edinburgh,Black History Walking Tour of Edinburgh,live,1595055,"[{'start_ts': '2020-09-12T10:30:00+01:00', 'en...","[{'type': 'description.official', 'description...",NaN,...,Days out,"{'dropin_event': False, 'booking_essential': F...",3,2,sample_20200501.json,2020-05-01,2020-11-01,1,NaN,NaN
2279,1599103,2020-09-24T14:24:20Z,2020-09-22T12:14:45Z,Pumpkin Picking,Pumpkin Picking,live,1599103,"[{'start_ts': '2020-10-16T09:00:00+01:00', 'en...","[{'type': 'description.list.default', 'descrip...",NaN,...,Days out,"{'dropin_event': False, 'booking_essential': F...",3,2,sample_20200501.json,2020-05-01,2020-11-01,1,{'info': '07793 600 289'},NaN


In [ ]:
type(df_new_events['schedules'])

pandas.core.series.Series

In [ ]:
df_events=df_new_events[['event_id','id', 'name', 'created_ts', 'modified_ts','website', 'ranking_in_level', 'ranking_level', 'sort_name', 'status']]

**Important** Use this dataframe for reading the properties of events in your knowlege graph. Not the df_new_events

In [ ]:
df_events.to_pickle(dataframes_final+"/df_events")

## 2. EVENTS_DESCRIPTION DATAFRAME

In [ ]:
df_new_events['descriptions'].iloc[10]

[{'description': 'Observational humour from the charismatic Canadian stand-up.',
  'type': 'description.list.default'},
 {'description': "The majestic mountain moose brings his thrilling new show 'Easy Tiger' out of the Fringe jungle and onto the world stage for his 3rd solo tour.",
  'type': 'description.official'}]

In [ ]:
df_new_events['descriptions']

0       [{'type': 'description.list.default', 'descrip...
1       [{'type': 'description.list.default', 'descrip...
2       [{'type': 'description.list.default', 'descrip...
3       [{'type': 'description.list.default', 'descrip...
4       [{'type': 'description.list.default', 'descrip...
                              ...                        
2272    [{'type': 'description.list.default', 'descrip...
2276    [{'type': 'description.list.default', 'descrip...
2278    [{'type': 'description.official', 'description...
2279    [{'type': 'description.list.default', 'descrip...
2280    [{'type': 'description.list.default', 'descrip...
Name: descriptions, Length: 38700, dtype: object

In [ ]:
df_new_events['descriptions'].iloc[1]

[{'description': 'Brilliant mix of English tradition and American music from the virtuoso fingerstyle guitarist and singer, a former BBC Radio 2 Folk Musician of the Year.',
  'type': 'description.list.default'},
 {'description': "Nominated for Musician of the Year and for Best Album with Vagrant Stanzas at the BBC Radio 2 Folk Awards 2014, Martin's shows are intense, eclectic, spellbinding and deeply moving. \n\nThere is no doubt that after 35 years as a professional musician Martin is better than ever. Widely acknowledged as one of the finest acoustic and slide guitar players in the world, his interpretations of traditional songs are masterpieces of storytelling. \n\nMartin has been nominated an astounding 31 times in the 15 years of the BBC Radio 2 Folk Awards -- more than any other performer -- with 12 nominations for Musician of The Year, which he has won twice. 2008 saw an incredible five nominations for 'Prodigal Son' and two wins, whilst in 2010 he had an unprecedented six nomi

In [ ]:
df_e_desc=df_new_events[['event_id','descriptions']].explode('descriptions')

In [ ]:
df_e_desc['descriptions'].iloc[18]

{'description': 'Observational humour from the charismatic Canadian stand-up.',
 'type': 'description.list.default'}

In [ ]:
df_e_desc['descriptions'].iloc[19]

{'description': "The majestic mountain moose brings his thrilling new show 'Easy Tiger' out of the Fringe jungle and onto the world stage for his 3rd solo tour.",
 'type': 'description.official'}

In [ ]:
df_e_desc=df_e_desc[df_e_desc['descriptions'].notna()]
df_e_desc['descriptions'].apply(pd.Series)

,type,description
0,description.list.default,Swedish trio combining acoustic instrumentatio...
1,description.list.default,Brilliant mix of English tradition and America...
1,description.official,Nominated for Musician of the Year and for Bes...
2,description.list.default,"Robbie Burns was funny, right? So toast the ba..."
3,description.list.default,"The Stand's spankingly good new talent night, ..."
...,...,...
2276,description.list.default,Tour starting at Edinburgh's The Elephant Hous...
2278,description.official,"The tour will be led by Lisa Williams, directo..."
2278,description.list.default,"The tour will be led by Lisa Williams, directo..."
2279,description.list.default,"Pull on your wellies, wrap up warm and come pi..."


In [ ]:
df_e_desc=df_new_events[['event_id','descriptions']].explode('descriptions')
df_e_desc=df_e_desc[df_e_desc['descriptions'].notna()]
df_e_desc=pd.concat([df_e_desc.drop(['descriptions'], axis=1),
                     df_e_desc['descriptions'].apply(pd.Series)], axis=1)
#df_e_desc=df_e_desc.drop(0, axis=1)
df_e_desc

,event_id,type,description
0,157884,description.list.default,Swedish trio combining acoustic instrumentatio...
1,194419,description.list.default,Brilliant mix of English tradition and America...
1,194419,description.official,Nominated for Musician of the Year and for Bes...
2,240818,description.list.default,"Robbie Burns was funny, right? So toast the ba..."
3,345866,description.list.default,"The Stand's spankingly good new talent night, ..."
...,...,...,...
2276,1586592,description.list.default,Tour starting at Edinburgh's The Elephant Hous...
2278,1595055,description.official,"The tour will be led by Lisa Williams, directo..."
2278,1595055,description.list.default,"The tour will be led by Lisa Williams, directo..."
2279,1599103,description.list.default,"Pull on your wellies, wrap up warm and come pi..."


In [ ]:
type(df_e_desc['type'])

pandas.core.series.Series

In [ ]:
#comment this line if you want to save the dataframe to file
df_e_desc.to_pickle(dataframes_final+"/df_event_description")

## 3. EVENTS_CATEGORY DATAFRAME

In [ ]:
df_new_events['category']

0          Music
1          Music
2         Comedy
3         Comedy
4         Comedy
          ...   
2272       Sport
2276    Days out
2278    Days out
2279    Days out
2280    Days out
Name: category, Length: 38700, dtype: object

In [ ]:
df_e_category=df_new_events[['event_id','category']]
df_e_category

,event_id,category
0,157884,Music
1,194419,Music
2,240818,Comedy
3,345866,Comedy
4,347164,Comedy
...,...,...
2272,1584208,Sport
2276,1586592,Days out
2278,1595055,Days out
2279,1599103,Days out


In [ ]:
len(pd.unique(df_e_category['category']))

15

In [ ]:
df_e_category.to_pickle(dataframes_final+"/df_event_category")

## 4. EVENTS_PROPERTIES DATAFRAME

In [ ]:
df_e_prop=df_new_events[['event_id','properties']]
df_e_prop=pd.concat([df_e_prop.drop(['properties'], axis=1), df_e_prop['properties'].apply(pd.Series)], axis=1)
df_e_prop

,event_id,actor,actor:sample,affiliate:getmein,affiliate:seatwave,author,awards:fringe-sustainable-practice:2015,awards:fringe-sustainable-practice:2017,booking_essential,cast,...,list:website:comments-enabled,list:website:comments-end-date,list:website:company,list:website:hitlisted,list:website:list-of-sites,organisation,pa:rating,place:capacity:max,simpleview:original:categories,writer
0,157884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,2013-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,194419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,2020-01-28 05:01:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,240818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,345866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,347164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,1584208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2276,1586592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2278,1595055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2279,1599103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_e_prop.to_pickle(dataframes_final+"/df_event_properties")

## 5. EVENTS_TAGS Dataframe

In [ ]:
df_e_tags=df_new_events[['event_id','tags']].explode('tags')
df_e_tags=df_e_tags[df_e_tags['tags'].notna()]

In [ ]:
len(pd.unique(df_e_tags['tags']))

2174

In [ ]:
df_e_tags.to_pickle(dataframes_final+"/df_event_tags")

## 6 EVENTS_PHONE_NUMBER DATAFRAME

In [ ]:
df_events_pn=df_new_events[['event_id', 'phone_numbers']]
df_events_pn=pd.concat([df_events_pn.drop(['phone_numbers'], axis=1), df_events_pn['phone_numbers'].apply(pd.Series)], axis=1)
df_events_pn=df_events_pn.drop(0, axis=1)
df_events_pn

,event_id,box_office,info
0,157884,NaN,NaN
1,194419,NaN,NaN
2,240818,NaN,NaN
3,345866,NaN,NaN
4,347164,NaN,NaN
...,...,...,...
2272,1584208,NaN,NaN
2276,1586592,NaN,0131 555 5558
2278,1595055,NaN,NaN
2279,1599103,NaN,07793 600 289


In [ ]:
df_events_pn.to_pickle(dataframes_final+"/df_event_phonenumber")

## 7. Schedules Dataframe 

In [ ]:
df_schedules=df_new_events[['event_id', 'schedules']].explode('schedules')
df_schedules_total=pd.concat([df_schedules.drop(['schedules'], axis=1), df_schedules['schedules'].apply(pd.Series)], axis=1)
df_schedules_total

,event_id,start_ts,end_ts,place_id,performances,performance_space,phone_numbers
0,157884,2018-04-26T20:00:00+01:00,2018-04-26T20:00:00+01:00,383,"[{'ts': '2018-04-26T20:00:00+01:00', 'duration...",NaN,NaN
1,194419,2018-03-10T19:30:00+00:00,2018-03-10T19:30:00+00:00,11092,"[{'ts': '2018-03-10T19:30:00+00:00', 'duration...",NaN,NaN
1,194419,2018-03-08T20:00:00+00:00,2018-03-08T20:00:00+00:00,11200,"[{'ts': '2018-03-08T20:00:00+00:00', 'links': ...",NaN,NaN
1,194419,2018-05-07T20:00:00+01:00,2018-05-07T20:00:00+01:00,386,"[{'ts': '2018-05-07T20:00:00+01:00', 'links': ...",NaN,NaN
2,240818,2018-01-24T20:30:00+00:00,2018-01-28T20:30:00+00:00,1,"[{'ts': '2018-01-24T20:30:00+00:00', 'links': ...",NaN,NaN
...,...,...,...,...,...,...,...
2272,1584208,2020-10-10T16:00:00+01:00,2020-10-10T16:00:00+01:00,127508,"[{'ts': '2020-10-10T16:00:00+01:00', 'duration...",NaN,NaN
2276,1586592,2020-08-12T08:00:00+01:00,2020-10-30T08:00:00+00:00,127571,"[{'ts': '2020-08-12T08:00:00+01:00', 'duration...",NaN,NaN
2278,1595055,2020-09-12T10:30:00+01:00,2020-10-24T10:30:00+01:00,127985,"[{'ts': '2020-09-12T10:30:00+01:00', 'duration...",NaN,NaN
2279,1599103,2020-10-16T09:00:00+01:00,2020-10-19T09:00:00+01:00,128231,"[{'ts': '2020-10-16T09:00:00+01:00', 'duration...",NaN,NaN


In [ ]:
df_schedules=df_schedules_total[['event_id', 'start_ts', 'end_ts', 'place_id', 'performance_space']]
df_schedules

,event_id,start_ts,end_ts,place_id,performance_space
0,157884,2018-04-26T20:00:00+01:00,2018-04-26T20:00:00+01:00,383,NaN
1,194419,2018-03-10T19:30:00+00:00,2018-03-10T19:30:00+00:00,11092,NaN
1,194419,2018-03-08T20:00:00+00:00,2018-03-08T20:00:00+00:00,11200,NaN
1,194419,2018-05-07T20:00:00+01:00,2018-05-07T20:00:00+01:00,386,NaN
2,240818,2018-01-24T20:30:00+00:00,2018-01-28T20:30:00+00:00,1,NaN
...,...,...,...,...,...
2272,1584208,2020-10-10T16:00:00+01:00,2020-10-10T16:00:00+01:00,127508,NaN
2276,1586592,2020-08-12T08:00:00+01:00,2020-10-30T08:00:00+00:00,127571,NaN
2278,1595055,2020-09-12T10:30:00+01:00,2020-10-24T10:30:00+01:00,127985,NaN
2279,1599103,2020-10-16T09:00:00+01:00,2020-10-19T09:00:00+01:00,128231,NaN


In [ ]:
df_schedules.to_pickle(dataframes_final+"/df_schedules")

## 8. Schedules _TAGS DATAFRAME ??? 

NOTE: SCHEDULES DO NOT HAVE TAGS - so that part of the ontology is wrong - you should correct the ontology. 

## 8. Schedules_PhoneNumber Dataframe

In [ ]:
df_schedules_pn=df_schedules_total[['event_id', 'start_ts', 'place_id','end_ts', 'phone_numbers']]
df_schedules_pn=df_schedules_pn[df_schedules_pn['phone_numbers'].notna()]
df_schedules_pn

,event_id,start_ts,place_id,end_ts,phone_numbers
65,58304,2018-04-29T19:30:00+01:00,610,2018-04-29T19:30:00+01:00,{'info': '01506 777666'}
100,834033,2020-01-24T21:00:00+00:00,377,2020-01-24T21:00:00+00:00,{'info': '0844 573 8455'}
145,943017,2018-04-08T11:30:00+01:00,379,2018-04-08T11:30:00+01:00,{'info': '0844 557 2686'}
182,345799,2019-08-17T20:00:00+01:00,381,2019-08-17T20:00:00+01:00,{'info': '0131 473 2000'}
339,557533,2018-09-29T19:30:00+01:00,383,2018-09-29T19:30:00+01:00,{'info': '01316682019'}
...,...,...,...,...,...
2898,1680444,2021-08-15T20:30:00+01:00,129818,2021-08-15T20:30:00+01:00,{'info': '0131 473 2000'}
2899,1680445,2021-08-14T20:30:00+01:00,129818,2021-08-14T20:30:00+01:00,{'info': '0131 473 2000'}
2900,1680449,2021-08-26T20:30:00+01:00,129818,2021-08-26T20:30:00+01:00,{'info': '0131 473 2000'}
2901,1680450,2021-08-17T20:30:00+01:00,129818,2021-08-17T20:30:00+01:00,{'info': '0131 473 2000'}


In [ ]:
df_schedules_pn=pd.concat([df_schedules_pn.drop(['phone_numbers'], axis=1), df_schedules_pn['phone_numbers'].apply(pd.Series)], axis=1)

df_schedules_pn

,event_id,start_ts,place_id,end_ts,info
65,58304,2018-04-29T19:30:00+01:00,610,2018-04-29T19:30:00+01:00,01506 777666
100,834033,2020-01-24T21:00:00+00:00,377,2020-01-24T21:00:00+00:00,0844 573 8455
145,943017,2018-04-08T11:30:00+01:00,379,2018-04-08T11:30:00+01:00,0844 557 2686
182,345799,2019-08-17T20:00:00+01:00,381,2019-08-17T20:00:00+01:00,0131 473 2000
339,557533,2018-09-29T19:30:00+01:00,383,2018-09-29T19:30:00+01:00,01316682019
...,...,...,...,...,...
2898,1680444,2021-08-15T20:30:00+01:00,129818,2021-08-15T20:30:00+01:00,0131 473 2000
2899,1680445,2021-08-14T20:30:00+01:00,129818,2021-08-14T20:30:00+01:00,0131 473 2000
2900,1680449,2021-08-26T20:30:00+01:00,129818,2021-08-26T20:30:00+01:00,0131 473 2000
2901,1680450,2021-08-17T20:30:00+01:00,129818,2021-08-17T20:30:00+01:00,0131 473 2000


In [ ]:
df_schedules_pn.to_pickle(dataframes_final+"/df_schedules_phone_numbers")

## 9. Performances Dataframe

In [ ]:
df_total_performances=df_schedules_total[['event_id','place_id', 'start_ts', 'end_ts', 'performances']].explode('performances')
df_total_performances=pd.concat([df_total_performances.drop(['performances'], axis=1), df_total_performances['performances'].apply(pd.Series)], axis=1)
#### NEW FOR DROPPING REPEATED PERFORMANCES
df_total_performances=df_total_performances.drop_duplicates(subset=['ts', 'place_id','start_ts', 'end_ts', 'event_id'], keep="first")
df_total_performances

,event_id,place_id,start_ts,end_ts,ts,duration,links,tickets,properties,descriptions,time_unknown
0,157884,383,2018-04-26T20:00:00+01:00,2018-04-26T20:00:00+01:00,2018-04-26T20:00:00+01:00,150.0,"[{'type': 'booking', 'url': 'http://www.theque...","[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,NaN,NaN
1,194419,11092,2018-03-10T19:30:00+00:00,2018-03-10T19:30:00+00:00,2018-03-10T19:30:00+00:00,120.0,NaN,"[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,NaN,NaN
1,194419,11200,2018-03-08T20:00:00+00:00,2018-03-08T20:00:00+00:00,2018-03-08T20:00:00+00:00,NaN,"[{'type': 'booking', 'url': 'https://www.ticke...","[{'type': 'Standard', 'currency': 'GBP', 'desc...",{'performance.sold-out': True},NaN,NaN
1,194419,386,2018-05-07T20:00:00+01:00,2018-05-07T20:00:00+01:00,2018-05-07T20:00:00+01:00,NaN,"[{'type': 'booking', 'url': 'https://www.trave...","[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,NaN,NaN
2,240818,1,2018-01-24T20:30:00+00:00,2018-01-28T20:30:00+00:00,2018-01-24T20:30:00+00:00,NaN,"[{'type': 'booking', 'url': 'http://www.thesta...","[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,"[{'type': 'list.description.default', 'descrip...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
2279,1599103,128231,2020-10-16T09:00:00+01:00,2020-10-19T09:00:00+01:00,2020-10-16T09:00:00+01:00,480.0,"[{'type': 'booking', 'url': 'https://www.kildu...","[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,NaN,NaN
2279,1599103,128231,2020-10-16T09:00:00+01:00,2020-10-19T09:00:00+01:00,2020-10-17T09:00:00+01:00,480.0,"[{'type': 'booking', 'url': 'https://www.kildu...","[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,NaN,NaN
2279,1599103,128231,2020-10-16T09:00:00+01:00,2020-10-19T09:00:00+01:00,2020-10-18T09:00:00+01:00,480.0,"[{'type': 'booking', 'url': 'https://www.kildu...","[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,NaN,NaN
2279,1599103,128231,2020-10-16T09:00:00+01:00,2020-10-19T09:00:00+01:00,2020-10-19T09:00:00+01:00,480.0,"[{'type': 'booking', 'url': 'https://www.kildu...","[{'type': 'Standard', 'currency': 'GBP', 'min_...",NaN,NaN,NaN


In [ ]:
df_performaces=df_total_performances[['event_id','place_id', 'start_ts', 'end_ts', 'ts', 'duration', 'time_unknown' ]]

In [ ]:
#comment this line if you want to save the dataframe to file
df_performaces.to_pickle(dataframes_final+"/df_performances")

## 10. PERFORMANCES_PROPERTIES DATAFRAME


**IMPORTANT** WE HAVE REALISED THAT WE DONT LONGER NEED THESE THREE NODES: PROPERTYEVENTS, THEATHRE AND FILM
ALL the properties of these 3 nodes are now PERFORMANCE_PROPERTY. 

The ontology should be updated to reflect this

Note: The follow cell takes 2 or 3 mintues to run

In [ ]:
df_p_prop_total=df_total_performances[['event_id', 'place_id','start_ts', 'end_ts', 'ts', 'properties']]
df_p_prop_total= df_p_prop_total[df_p_prop_total['properties'].notna()]
df_p_prop=pd.concat([df_p_prop_total.drop(['properties'], axis=1), df_p_prop_total['properties'].apply(pd.Series)], axis=1)


In [ ]:
df_p_prop.drop_duplicates(keep='first')

In [ ]:
df_p_prop.to_pickle(dataframes_final+"/df_performances_properties")

## 11. PERFORMANCES DESCRIPTION

In [ ]:
df_p_desc=df_total_performances[['event_id', 'start_ts','place_id', 'end_ts', 'ts','descriptions']].explode('descriptions')
df_p_desc=df_p_desc[df_p_desc['descriptions'].notna()]
df_p_desc=pd.concat([df_p_desc.drop(['descriptions'], axis=1), df_p_desc['descriptions'].apply(pd.Series)], axis=1)
df_p_desc

In [ ]:
df_p_desc.drop_duplicates(keep='first')

In [ ]:
df_p_desc.to_pickle(dataframes_final+"/df_performances_descriptions")

## 12. PERFORMANCE LINKS

These lines takes 5 minutes to run

In [ ]:
df_p_links=df_total_performances[['event_id', 'start_ts', 'place_id','end_ts', 'ts','links']].explode('links')
df_p_links=df_p_links[df_p_links['links'].notna()]
df_p_links=pd.concat([df_p_links.drop(['links'], axis=1), df_p_links['links'].apply(pd.Series)], axis=1)
df_p_links

In [ ]:
df_p_links.to_pickle(dataframes_final+"/df_performances_links")

## 13. TICKETS

This cell takes 5 minutes to run


In [ ]:
df_tickets=df_total_performances[['event_id','place_id','start_ts', 'end_ts', 'ts','tickets']].explode('tickets')
df_tickets=df_tickets[df_tickets['tickets'].notna()]
df_tickets
df_tickets=pd.concat([df_tickets.drop(['tickets'], axis=1), df_tickets['tickets'].apply(pd.Series)], axis=1)


In [ ]:
df_tickets

## 14. Ticket description dataframe

In [ ]:
df_ticketDescription=df_tickets[['event_id','place_id','start_ts', 'type','end_ts', 'ts','description']]

df_ticketsDescription= df_ticketDescription[df_ticketDescription['description'].notna()]
df_ticketsDescription

In [ ]:
df_ticketsDescription=df_ticketsDescription.drop_duplicates(keep='first')
df_ticketsDescription

In [ ]:
df_ticketsDescription.to_pickle(dataframes_final+"/df_ticketDescription")

In [ ]:
df_tickets=df_tickets.drop(['description'],axis=1)
df_tickets

In [ ]:
df_tickets=df_tickets.drop_duplicates(keep='first')
df_tickets

In [ ]:
# un-comment this line if you want to save the df_p_properties into a file
df_tickets.to_pickle(dataframes_final+"/df_tickets")

## 14. PLACES

### Replace the location with the location of downloaded copy of df_places from google drive.

In [ ]:
with open("/content/drive/MyDrive/df_places","rb") as df_places:
    df_places_total=pickle.load(df_places)

In [ ]:
df_places_total.iloc[0]

In [ ]:
df_places=df_places_total[['place_id', 'created_ts', 'modified_ts', 'name', 'sort_name', 'address', 'town', 'postal_code', 'country_code', 'website', 'email', 'status']]
df_places

In [ ]:
df_places.to_pickle(dataframes_final+"/df_places")

## 15. PLACES PROPERTIES DATAFRAME

In [ ]:
df_place_prop=df_places_total[['place_id','properties']]
df_place_prop=df_place_prop[df_place_prop['properties'].notna()]
df_place_prop=pd.concat([df_place_prop.drop(['properties'], axis=1), df_place_prop['properties'].apply(pd.Series)], axis=1)
df_place_prop

In [ ]:

df_place_prop.to_pickle(dataframes_final+"/df_places_properties")

## 16. PLACE PHONE NUMBER

In [ ]:
df_places_pn=df_places_total[['place_id','phone_numbers']]
df_places_pn=df_places_pn[df_places_pn['phone_numbers'].notna()]
df_places_pn=pd.concat([df_places_pn.drop(['phone_numbers'], axis=1), df_places_pn['phone_numbers'].apply(pd.Series)], axis=1)

df_places_pn

In [ ]:
df_places_pn.to_pickle(dataframes_final+"/df_places_pn")

## 17. PLACE LOCATION DATAFRAME

In [ ]:
df_places_loc=df_places_total[['place_id','loc']]
df_places_loc=df_places_loc[df_places_loc['loc'].notna()]
df_places_loc=pd.concat([df_places_loc.drop(['loc'], axis=1), df_places_loc['loc'].apply(pd.Series)], axis=1)

df_places_loc

In [ ]:

df_places_loc.to_pickle(dataframes_final+"/df_places_loc")

## 18. PLACES DESCRIPTION

In [ ]:
df_places_desc=df_places_total[['place_id','descriptions']].explode('descriptions')
df_places_desc=df_places_desc[df_places_desc['descriptions'].notna()]
df_places_desc=pd.concat([df_places_desc.drop(['descriptions'], axis=1), df_places_desc['descriptions'].apply(pd.Series)], axis=1)

df_places_desc

In [ ]:
# un-comment this line if you want to save the df_p_properties into a file
df_places_desc.to_pickle(dataframes_final+"/df_places_description")

## 19. PLACES TAGS

In [ ]:
df_places_tags=df_places_total[['place_id','tags']].explode('tags')
df_places_tags

In [ ]:
df_places_tags=df_places_tags[df_places_tags['tags'].notna()]

In [ ]:
len(pd.unique(df_places_tags.tags))

In [ ]:
df_places_tags.to_pickle(dataframes_final+"/df_places_tags")

In [ ]:
df_places_tags

# SPARQL Vereification Data Set

In [ ]:
CheckData=df_schedules.merge(df_places,on=['place_id','place_id'])

In [ ]:
CheckData=CheckData.merge(df_e_tags,on=['event_id'])

In [ ]:
CheckData=CheckData.merge(df_e_category,on=['event_id'])

In [ ]:
Edinburgh=CheckData[CheckData['town']=='Edinburgh']

In [ ]:
tem=Edinburgh.loc[Edinburgh['tags']=='Music']

In [ ]:
tem[tem['category']=='Music']

In [ ]:
def dataframe_groupby_size(df, column_list, rename, level=None, city=None, period="full"):
    if len(column_list)==1:
        column=column_list[0]
        df_v1=df.groupby([column]).size().reset_index()
        df_v1=df_v1.rename(columns={0: rename}).sort_values(by=[rename], ascending=False)
        if city:
            title= level+" " + rename+ " per "+ column + " at " +city
            if period!= "full":
                title = title + " for the month of " + month_string + " over the years"
            fig_scatter=px.scatter(df_v1, x=column,y=rename, color=rename, size=rename, size_max=50, title=title)
            fig_bar= px.bar(df_v1, x=column, y=rename, color=column, barmode='group', title=title)
            
           
        else:
            title= level+" " + rename+ " per "+ column 
            if period != "full":
                title = title + " for the month of " + month_string + " over the years"
            fig_scatter=px.scatter(df_v1, x=column,y=rename, color=rename, size=rename, size_max=50, title=title)
            fig_bar= px.bar(df_v1, x=column, y=rename, color=column, barmode='group', title=title)
            

        return df_v1, fig_scatter, fig_bar
    else:
        df_v1=df.groupby(column_list).size().reset_index()
        df_v1=df_v1.rename(columns={0: rename})
        return df_v1

In [ ]:
g_tags=dataframe_groupby_size(tem, ['tags','category'], 'frequency', 'Events')

In [ ]:
g_tags